<center><h1> FLamby integration into Fed-BioMed </h1></center>

In [ ]:
%load_ext autoreload
%autoreload 2

This notebook highlights practical use cases for the FLamby integration into Fed-BioMed. A few datasets are here tested:
1. IXI Tiny
2. Heart-Disease
3. TCGA-BRCA
4. Synthetic
5. ISIC

For one FLamby dataset (LIDC-IDRI), we noticed conflicts over few packages including Torch, which cannot be easily resolved by changing package versions but need changes in the code (either on FLamby or Fed-BioMed side). This situation shows a certain complexity regarding the interoperability and has to be taken into account for future datasets that will be part of the suite.

## 1. Fed-IXI

In [ ]:
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import AdamW

from flamby.datasets.fed_ixi import (Baseline as ixi_baseline,
                                     BaselineLoss as ixi_baseline_loss,
                                     Optimizer as ixi_optimizer,
                                     BATCH_SIZE as ixi_batch_size,
                                     LR as ixi_lr)

class UNetTrainingPlan(TorchTrainingPlan):
    # Init of UNetTrainingPlan
    def __init__(self, model_args: dict = {}):
        super(UNetTrainingPlan, self).__init__(model_args)
        
        self.model = ixi_baseline() # UNet model
        self.loss = ixi_baseline_loss() # Dice loss
        
        deps = ['from torch.optim import AdamW',
               'from flamby.datasets.fed_ixi import (Baseline as ixi_baseline,\
                BaselineLoss as ixi_baseline_loss,\
                Optimizer as ixi_optimizer)',]
        self.add_dependency(deps)
        
        self.optimizer = ixi_optimizer(self.parameters()) # AdamW
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, img, target):
        #this function must return the loss to backward it 
        output = self.forward(img)
        loss = self.loss(output, target)
        return loss

In [ ]:
training_args = {
    'batch_size': ixi_batch_size,
    'lr': ixi_lr,
    'epochs': 1,
}

*train_transform_flamby* key in **training_args** can optionally be used to perform extra transformations on a flamby dataset.

As a reminder, flamby datasets are already internally handling a transformation through their dataloader (this internal transform
is the one officially used for the flamby benchmark). Thus, one should check what is already performed on the flamby side before
adding transforms through the researcher.

*train_transform_flamby* has to be defined as a list containing two elements:
- the first is the imports needed to perform the transformation
- the second is the Compose object that will be used to input the transform parameter of the flamby dataset federated class

Example:
```python
training_args = {
    ...,
    'train_transform_flamby':["from monai.transforms import (Compose, NormalizeIntensity, Resize,)",
                         "Compose([Resize((48,60,48)), NormalizeIntensity()])"]
}
```

### Train the federated model

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['ixi']
num_rounds = 1

exp1 = Experiment(tags=tags,
                 model_class=UNetTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

An experiment is a class that orchestrates the training processes that run on different nodes. The experiment has been here initialized with necessary arguments to inform nodes about how to process the training data based on a given model.

Let's run the experiment. According to the provided arguments, 1 training round should be completed on the nodes that you created (3 nodes here because there are 3 centers in the case of IXI).
Aggregated parameters are saved at the end of the experiment, and their state at every round can be loaded.

In [ ]:
exp1.run()

Retrieve the federated model parameters

In [ ]:
fed_model_ixi = exp1.model_instance()
fed_model_ixi.load_state_dict(exp1.aggregated_params()[num_rounds - 1]['params'])

### Test the federated model

In [ ]:
from torch.utils.data import DataLoader
from flamby.utils import evaluate_model_on_tests
from flamby.datasets.fed_ixi import (metric as ixi_metric,
                                     FedClass as ixi_fed)

In [ ]:
test_dataloader_ixi_pooled = DataLoader(dataset=ixi_fed(train=False, pooled=True),batch_size=ixi_batch_size)
test_dataloader_ixi_client0 = DataLoader(dataset=ixi_fed(center=0, train=False),batch_size=ixi_batch_size)
test_dataloader_ixi_client1 = DataLoader(dataset=ixi_fed(center=1, train=False),batch_size=ixi_batch_size)
test_dataloader_ixi_client2 = DataLoader(dataset=ixi_fed(center=2, train=False),batch_size=ixi_batch_size)

In [ ]:
evaluate_model_on_tests(fed_model_ixi,
                        [test_dataloader_ixi_pooled,
                         test_dataloader_ixi_client0,
                         test_dataloader_ixi_client1,
                         test_dataloader_ixi_client2],
                        ixi_metric)

## 2. Fed-Heart-Disease

In [ ]:
from flamby.datasets.fed_heart_disease import (Baseline as hd_baseline,
                                               BaselineLoss as hd_baseline_loss,
                                               Optimizer as hd_optimizer,
                                               BATCH_SIZE as hd_batch_size,
                                               LR as hd_lr)

from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import Adam

class FedHeartTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(FedHeartTrainingPlan, self).__init__(model_args)
        
        self.model = hd_baseline()
        self.loss = hd_baseline_loss()
        
        deps = ['from torch.optim import Adam',
                'from flamby.datasets.fed_heart_disease import (Baseline as hd_baseline,\
                BaselineLoss as hd_baseline_loss,\
                Optimizer as hd_optimizer)']
        self.add_dependency(deps)
        
        self.optimizer = hd_optimizer(self.parameters())
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss = self.loss(output, target)
        return loss

In [ ]:
training_args = {
    'batch_size': hd_batch_size,
    'lr': hd_lr,
    'epochs': 5,
}

### Train the federated model

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['hd']
num_rounds = 1

exp2 = Experiment(tags=tags,
                 model_class=FedHeartTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

In [ ]:
exp2.run()

Retrieve the federated model parameters

In [ ]:
fed_model_hd = exp2.model_instance()
fed_model_hd.load_state_dict(exp2.aggregated_params()[num_rounds - 1]['params'])

### Test the federated model

In [ ]:
from flamby.datasets.fed_heart_disease import (metric as hd_metric,
                                               FedClass as hd_fed)

test_dataloader_hd_pooled = DataLoader(dataset=hd_fed(train=False, pooled=True),batch_size=hd_batch_size)
test_dataloader_hd_client0 = DataLoader(dataset=hd_fed(center=0, train=False),batch_size=hd_batch_size)
test_dataloader_hd_client1 = DataLoader(dataset=hd_fed(center=1, train=False),batch_size=hd_batch_size)
test_dataloader_hd_client2 = DataLoader(dataset=hd_fed(center=2, train=False),batch_size=hd_batch_size)
test_dataloader_hd_client3 = DataLoader(dataset=hd_fed(center=3, train=False),batch_size=hd_batch_size)

In [ ]:
evaluate_model_on_tests(fed_model_hd,
                        [test_dataloader_hd_pooled,
                         test_dataloader_hd_client0,
                         test_dataloader_hd_client1,
                         test_dataloader_hd_client2,
                         test_dataloader_hd_client3],
                        hd_metric)

## 3. Fed-TCGA-BRCA

In [ ]:
from flamby.datasets.fed_tcga_brca import (Baseline as tc_baseline,
                                           BaselineLoss as tc_baseline_loss,
                                           Optimizer as tc_optimizer,
                                           BATCH_SIZE as tc_batch_size,
                                           LR as tc_lr)
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import Adam

class FedTcgaBrcaTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(FedTcgaBrcaTrainingPlan, self).__init__(model_args)
        
        self.model = tc_baseline()
        self.loss = tc_baseline_loss()
        
        deps = ['from torch.optim import Adam',
               'from flamby.datasets.fed_tcga_brca import (Baseline as tc_baseline,\
                BaselineLoss as tc_baseline_loss,\
                Optimizer as tc_optimizer)']
        self.add_dependency(deps)
        
        self.optimizer = tc_optimizer(self.parameters())
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss = self.loss(output, target)
        return loss

In [ ]:
training_args = {
    'batch_size': tc_batch_size,
    'lr': tc_lr,
    'epochs': 5,
}

### Train the federated model

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['tc']
num_rounds = 1

exp3 = Experiment(tags=tags,
                 model_class=FedTcgaBrcaTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

In [ ]:
exp3.run()

Retrieve the federated model parameters

In [ ]:
fed_model_tc = exp3.model_instance()
fed_model_tc.load_state_dict(exp3.aggregated_params()[num_rounds - 1]['params'])

### Test the federated model

In [ ]:
from flamby.datasets.fed_tcga_brca import (metric as tc_metric,
                                           FedClass as tc_fed)

test_dataloader_tc_pooled = DataLoader(dataset=tc_fed(train=False, pooled=True),batch_size=tc_batch_size)
test_dataloader_tc_client0 = DataLoader(dataset=tc_fed(center=0, train=False),batch_size=tc_batch_size)
test_dataloader_tc_client1 = DataLoader(dataset=tc_fed(center=1, train=False),batch_size=tc_batch_size)
test_dataloader_tc_client2 = DataLoader(dataset=tc_fed(center=2, train=False),batch_size=tc_batch_size)
test_dataloader_tc_client3 = DataLoader(dataset=tc_fed(center=3, train=False),batch_size=tc_batch_size)
test_dataloader_tc_client4 = DataLoader(dataset=tc_fed(center=4, train=False),batch_size=tc_batch_size)
test_dataloader_tc_client5 = DataLoader(dataset=tc_fed(center=5, train=False),batch_size=tc_batch_size)

In [ ]:
evaluate_model_on_tests(fed_model_tc,
                        [test_dataloader_tc_pooled,
                         test_dataloader_tc_client0,
                         test_dataloader_tc_client1,
                         test_dataloader_tc_client2,
                         test_dataloader_tc_client3,
                         test_dataloader_tc_client4,
                         test_dataloader_tc_client5],
                        tc_metric)

## 4. Fed-Synthetic

In [ ]:
from flamby.datasets.fed_synthetic import (Baseline as sy_baseline,
                                           BaselineLoss as sy_baseline_loss,
                                           Optimizer as sy_optimizer,
                                           BATCH_SIZE as sy_batch_size,
                                           LR as sy_lr)
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import Adam

class FedSyntheticTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(FedSyntheticTrainingPlan, self).__init__(model_args)
        
        self.model = sy_baseline(model_args.get('input_dim', 10), model_args.get('output_dim', 1)) # specific to synthetic use case
        self.loss = sy_baseline_loss()
        
        deps = ['from torch.optim import Adam',
               'from flamby.datasets.fed_synthetic import (Baseline as sy_baseline,\
                BaselineLoss as sy_baseline_loss,\
                Optimizer as sy_optimizer)',]
        self.add_dependency(deps)
        
        self.optimizer = sy_optimizer(self.parameters())
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, data, target):
        #this function must return the loss to backward it 
        output = self.forward(data)
        loss = self.loss(output, target)
        return loss

In [ ]:
training_args = {
    'batch_size': sy_batch_size,
    'lr': sy_lr,
    'epochs': 5,
}

model_args = {
    'input_dim': 10,
    'output_dim': 1,
}

### Train the federated model

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['sy']
num_rounds = 1

exp4 = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=FedSyntheticTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

In [ ]:
exp4.run()

Retrieve the federated model parameters

In [ ]:
fed_model_sy = exp4.model_instance()
fed_model_sy.load_state_dict(exp4.aggregated_params()[num_rounds - 1]['params'])

### Test the federated model

In [ ]:
from flamby.datasets.fed_synthetic import (metric as sy_metric,
                                           FedClass as sy_fed)

test_dataloader_sy_pooled = DataLoader(dataset=sy_fed(train=False, pooled=True),batch_size=sy_batch_size)
test_dataloader_sy_client0 = DataLoader(dataset=sy_fed(center=0, train=False),batch_size=sy_batch_size)
test_dataloader_sy_client1 = DataLoader(dataset=sy_fed(center=1, train=False),batch_size=sy_batch_size)
test_dataloader_sy_client2 = DataLoader(dataset=sy_fed(center=2, train=False),batch_size=sy_batch_size)
test_dataloader_sy_client3 = DataLoader(dataset=sy_fed(center=3, train=False),batch_size=sy_batch_size)

In [ ]:
evaluate_model_on_tests(fed_model_sy,
                        [test_dataloader_sy_pooled,
                         test_dataloader_sy_client0,
                         test_dataloader_sy_client1,
                         test_dataloader_sy_client2,
                         test_dataloader_sy_client3],
                        sy_metric)

## 5. Fed-ISIC

In [ ]:
from flamby.datasets.fed_isic2019 import (Baseline as is_baseline,
                                          BaselineLoss as is_baseline_loss,
                                          Optimizer as is_optimizer,
                                          BATCH_SIZE as is_batch_size,
                                          LR as is_lr)
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import Adam

class FedISICTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(FedISICTrainingPlan, self).__init__(model_args)
        
        self.model = is_baseline()
        self.loss = is_baseline_loss()
        
        deps = ['from torch.optim import Adam',
               'from flamby.datasets.fed_isic2019 import (Baseline as is_baseline,\
                BaselineLoss as is_baseline_loss,\
                Optimizer as is_optimizer)',]
        self.add_dependency(deps)
        
        self.optimizer = Adam(self.parameters())
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss = self.loss(output, target)
        return loss

In [ ]:
training_args = {
    'batch_size': is_batch_size,
    'lr': is_lr,
    'epochs': 1,
}

### Train the federated model

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['is']
num_rounds = 1

exp5 = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=FedISICTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

In [ ]:
exp5.run()

Retrieve the federated model parameters

In [ ]:
fed_model_is = exp5.model_instance()
fed_model_is.load_state_dict(exp5.aggregated_params()[num_rounds - 1]['params'])

### Test the federated model

In [ ]:
from flamby.datasets.fed_isic2019 import (metric as is_metric,
                                          FedClass as is_fed)

test_dataloader_is_pooled = DataLoader(dataset=is_fed(train=False, pooled=True),batch_size=is_batch_size)
test_dataloader_is_client0 = DataLoader(dataset=is_fed(center=0, train=False),batch_size=is_batch_size)
test_dataloader_is_client1 = DataLoader(dataset=is_fed(center=1, train=False),batch_size=is_batch_size)
test_dataloader_is_client2 = DataLoader(dataset=is_fed(center=2, train=False),batch_size=is_batch_size)
test_dataloader_is_client3 = DataLoader(dataset=is_fed(center=3, train=False),batch_size=is_batch_size)
test_dataloader_is_client4 = DataLoader(dataset=is_fed(center=4, train=False),batch_size=is_batch_size)
test_dataloader_is_client5 = DataLoader(dataset=is_fed(center=5, train=False),batch_size=is_batch_size)

In [ ]:
evaluate_model_on_tests(fed_model_is,
                        [test_dataloader_is_pooled,
                         test_dataloader_is_client0,
                         test_dataloader_is_client1,
                         test_dataloader_is_client2,
                         test_dataloader_is_client3,
                         test_dataloader_is_client4,
                         test_dataloader_is_client5],
                        is_metric)